In [ ]:
from asgardpy.analysis import AsgardpyAnalysis
from asgardpy.config import AsgardpyConfig

from gammapy.data import DataStore
from gammapy.makers import SpectrumDatasetMaker

import matplotlib.pyplot as plt
import numpy as np

import astropy.units as u
import logging

In [ ]:
from asgardpy.data.dataset_1d import Dataset1DGeneration
from asgardpy.io.io import DL3Files

In [ ]:
log = logging.getLogger("test 1d dataset")

In [ ]:
config_file = "/path/to/Config.yaml"

In [ ]:
config = AsgardpyConfig()

In [ ]:
%%time
config_main = config.read(config_file)

In [ ]:
for g in config_main.general:
    print(g)

# Steps mentioned in the main config file

In [ ]:
config_main.general.steps

# Target source information

In [ ]:
for c in config_main.target:
    print(c)

# 1D Datasets informations

In [ ]:
for cc in config_main.dataset1d:
    print(cc)

In [ ]:
%%time
analysis = AsgardpyAnalysis(config_main)

In [ ]:
analysis

# Prepare for a single 1D Dataset, based on instrument

In [ ]:
instruments_list = config_main.dataset1d.instruments
print(len(instruments_list), "number of 1D dataset information provided")

config_1d_dataset = instruments_list[0]
print(f"Instrument selected is {config_1d_dataset.name}")

# Run each Dataset1DGeneration step

In [ ]:
%%time
generate_1d_dataset = Dataset1DGeneration(
    log, config_1d_dataset, config_main.target
)

# Initialized variables

In [ ]:
print(generate_1d_dataset.config_1d_dataset_io)

In [ ]:
print(generate_1d_dataset.config_1d_dataset_info)

In [ ]:
print(generate_1d_dataset.config_target)

### First check for the given file list if they are readable or not.

In [ ]:
%%time
file_list = {}
dl3_info = DL3Files(
    generate_1d_dataset.config_1d_dataset_io[0], 
    file_list, 
    log=generate_1d_dataset.log
)

In [ ]:
dl3_info

In [ ]:
%%time
dl3_info.list_dl3_files()

In [ ]:
print(f"Reading from {len(dl3_info.events_files)} DL3 files")
for e in dl3_info.events_files:
    print(e)

## Get Observations object

In [ ]:
datastore = DataStore.from_dir(generate_1d_dataset.config_1d_dataset_io[0].input_dir)

In [ ]:
datastore.obs_table

In [ ]:
observations = generate_1d_dataset.get_filtered_observations(datastore)

In [ ]:
print(observations)

In [ ]:
print("Using the list of required IRFs:", generate_1d_dataset.config_1d_dataset_info.observation.required_irfs)
print("Selected observations from the given DataStore:", observations.ids)

## Create the base dataset template and get all the Dataset Reduction makers 

In [ ]:
dataset_template = generate_1d_dataset.generate_geom()

In [ ]:
print(dataset_template)

In [ ]:
%%time
dataset_maker = SpectrumDatasetMaker(
    containment_correction=generate_1d_dataset.config_1d_dataset_info.containment_correction,
    selection=generate_1d_dataset.config_1d_dataset_info.map_selection,
)

In [ ]:
%%time
bkg_maker = generate_1d_dataset.get_bkg_maker()

In [ ]:
%%time
safe_maker = generate_1d_dataset.get_safe_mask_maker()

In [ ]:
print(dataset_maker)

In [ ]:
print(bkg_maker)
print(bkg_maker.region_finder)
print("Number of Wobble OFF regions selected:", bkg_maker.region_finder.n_off_regions)

In [ ]:
print(safe_maker)

In [ ]:
%%time
generate_1d_dataset.generate_dataset(
    observations,
    dataset_template,
    dataset_maker,
    bkg_maker,
    safe_maker
)

In [ ]:
print(generate_1d_dataset.datasets)

In [ ]:
info_table = generate_1d_dataset.datasets.info_table(cumulative=True)

In [ ]:
stacked_dataset = generate_1d_dataset.datasets.stack_reduce(name=config_1d_dataset.name)

In [ ]:
print(stacked_dataset)

# Some Basic plots from the Dataset

In [ ]:
info_table

In [ ]:
# Plot temporal evolution of excess events and significance value
plt.figure(figsize=(10,5))
plt.subplot(121)
plt.plot(
    np.sqrt(info_table["livetime"].to("h")), info_table["excess"], marker="o", ls="none"
)
plt.plot(info_table["livetime"].to("h")[-1:1], info_table["excess"][-1:1], 'r')
plt.xlabel("Sqrt Livetime h^(1/2)")
plt.ylabel("Excess")
plt.grid()
plt.title('Excess vs Square root of Livetime')

plt.subplot(122)
plt.plot(
    np.sqrt(info_table["livetime"].to("h")),
    info_table["sqrt_ts"],
    marker="o",
    ls="none",
)
plt.grid()
plt.xlabel("Sqrt Livetime h^(1/2)")
plt.ylabel("sqrt_ts")
plt.title('Significance vs Square root of Livetime')
plt.subplots_adjust(wspace=0.5)

In [ ]:
stacked_dataset.peek()